In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from Transformer import ReplaceZeroWithMean
import joblib


RFC_URL = "Data/Models/RFC"
SVM_URL = "Data/Models/SVM"

models = {
    "RFC": f"{RFC_URL}/RFC_no_hyper.pkl",
    "RFC_hyper": f"{RFC_URL}/RFC_hyper.pkl",
    "RFC_hyper_cv": f"{RFC_URL}/RFC_hyper_cv.pkl",
    "RFC_hyper_cv_tuned": f"{RFC_URL}/RFC_hyper_cv_tuned.pkl",
    # "SVM": f"{SVM_URL}/SVM_no_hyper.pkl",
    # "SVM_para_grid": f"{SVM_URL}/SVM_para_grid.pkl",
    # "SVM_para_sampl": f"{SVM_URL}/SVM_para_sample.pkl",
}


data = pd.read_csv("./Data/diabetes.csv")
print("Lable Balance -> ", (data["Outcome"] == 1).sum(), "|", (data["Outcome"] == 0).sum())
print()
print("Features Zero\\NaN -> \n", data.drop("Outcome", axis=1).where(data == 0).count(axis=0), sep="")

Lable Balance ->  268 | 500

Features Zero\NaN -> 
Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
dtype: int64


In [4]:
from pprint import pprint


X = data.drop("Outcome", axis=1)
y = data["Outcome"]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, random_state=1)
imputer = ReplaceZeroWithMean(["Glucose", "BloodPressure", "SkinThickness", "BMI"])
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

scores = {}
for name, model in models.items():
    model = joblib.load(model)
    model.fit(X_train, y_train)
    scores.update({name: model.score(X_test, y_test)})

pprint(scores)

{'RFC': 0.8051948051948052,
 'RFC_hyper': 0.8571428571428571,
 'RFC_hyper_cv': 0.8311688311688312,
 'RFC_hyper_cv_tuned': 0.8376623376623377}
